In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score

# Set random seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

set_seed()

# Dummy dataset for multimodal graph
class MultimodalGraphDataset(Dataset):
    def __init__(self, num_samples=1000, feature_dim=128):
        super(MultimodalGraphDataset, self).__init__()
        self.num_samples = num_samples
        self.feature_dim = feature_dim
        
        # Generate dummy features for text, images, and speech modalities
        self.text_features = np.random.rand(num_samples, feature_dim)
        self.image_features = np.random.rand(num_samples, feature_dim)
        self.speech_features = np.random.rand(num_samples, feature_dim)
        
        # Labels for missing link prediction
        self.labels = np.random.randint(0, 2, size=num_samples)

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        return (
            torch.tensor(self.text_features[idx], dtype=torch.float32),
            torch.tensor(self.image_features[idx], dtype=torch.float32),
            torch.tensor(self.speech_features[idx], dtype=torch.float32),
            torch.tensor(self.labels[idx], dtype=torch.float32)
        )

# Contrastive loss for MGCL
class ContrastiveLoss(nn.Module):
    def __init__(self, temperature=0.5):
        super(ContrastiveLoss, self).__init__()
        self.temperature = temperature
        self.cosine_similarity = nn.CosineSimilarity(dim=-1)

    def forward(self, anchor, positive, negatives):
        pos_similarity = self.cosine_similarity(anchor, positive) / self.temperature
        neg_similarity = self.cosine_similarity(anchor.unsqueeze(1), negatives) / self.temperature
        
        exp_pos = torch.exp(pos_similarity)
        exp_neg = torch.sum(torch.exp(neg_similarity), dim=1)
        
        loss = -torch.log(exp_pos / (exp_pos + exp_neg))
        return torch.mean(loss)

# Multimodal Graph Contrastive Learning Model
class MultimodalGraphModel(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(MultimodalGraphModel, self).__init__()
        
        # Separate encoders for each modality
        self.text_encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU()
        )
        self.image_encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU()
        )
        self.speech_encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU()
        )

        # Fusion layer
        self.fusion_layer = nn.Sequential(
            nn.Linear(hidden_dim * 3, hidden_dim),
            nn.ReLU()
        )

        # Prediction layer for missing link
        self.predictor = nn.Sequential(
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )

    def forward(self, text, image, speech):
        text_embedding = self.text_encoder(text)
        image_embedding = self.image_encoder(image)
        speech_embedding = self.speech_encoder(speech)

        # Fuse embeddings
        fused_embedding = torch.cat((text_embedding, image_embedding, speech_embedding), dim=-1)
        fused_embedding = self.fusion_layer(fused_embedding)

        # Predict missing link
        prediction = self.predictor(fused_embedding)
        return fused_embedding, prediction

# Training loop
def train(model, train_loader, criterion, optimizer, contrastive_loss, device):
    model.train()
    total_loss = 0
    for text, image, speech, labels in train_loader:
        text, image, speech, labels = text.to(device), image.to(device), speech.to(device), labels.to(device)
        
        # Forward pass
        embeddings, predictions = model(text, image, speech)
        
        # Generate contrastive loss inputs (dummy for demonstration)
        positives = embeddings
        negatives = embeddings[torch.randperm(embeddings.size(0))]
        
        # Compute losses
        pred_loss = criterion(predictions.squeeze(), labels)
        cont_loss = contrastive_loss(embeddings, positives, negatives)
        loss = pred_loss + cont_loss
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(train_loader)

# Evaluation loop
def evaluate(model, data_loader, device):
    model.eval()
    all_labels = []
    all_preds = []
    
    with torch.no_grad():
        for text, image, speech, labels in data_loader:
            text, image, speech, labels = text.to(device), image.to(device), speech.to(device), labels.to(device)
            _, predictions = model(text, image, speech)
            
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predictions.squeeze().cpu().numpy())

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    accuracy = accuracy_score(all_labels, all_preds > 0.5)
    precision = precision_score(all_labels, all_preds > 0.5, average='macro')
    auc = roc_auc_score(all_labels, all_preds)

    return accuracy, precision, auc

# Main
if __name__ == "__main__":
    # Parameters
    input_dim = 128
    hidden_dim = 64
    num_epochs = 200
    batch_size = 32
    learning_rate = 0.006

    # Device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Dataset and DataLoader
    dataset = MultimodalGraphDataset()
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    # Model, Loss, Optimizer
    model = MultimodalGraphModel(input_dim, hidden_dim).to(device)
    criterion = nn.BCELoss()
    contrastive_loss = ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(num_epochs):
        train_loss = train(model, train_loader, criterion, optimizer, contrastive_loss, device)
        accuracy, precision, auc = evaluate(model, test_loader, device)

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, AUC: {auc:.4f}")


C:\Users\uclab1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 1/200, Loss: 3.8248, Accuracy: 0.5000, Precision: 0.2500, AUC: 0.5154


C:\Users\uclab1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 2/200, Loss: 4.1128, Accuracy: 0.5000, Precision: 0.2500, AUC: 0.5099


C:\Users\uclab1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

C:\Users\uclab1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 198/200, Loss: 4.1491, Accuracy: 0.5000, Precision: 0.2500, AUC: 0.5000


C:\Users\uclab1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 199/200, Loss: 4.1493, Accuracy: 0.5000, Precision: 0.2500, AUC: 0.5000
Epoch 200/200, Loss: 4.1492, Accuracy: 0.5000, Precision: 0.2500, AUC: 0.5000


C:\Users\uclab1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
